In [1]:
from julia import Julia

julia = Julia(compiled_modules=False)
from julia import Main
from julia.tools import redirect_output_streams

redirect_output_streams()


In [2]:
import pysr

In [3]:
import sympy
import numpy as np
from matplotlib import pyplot as plt
from pysr import PySRRegressor
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split

In [4]:
import os
cwd = os.getcwd()
print(cwd)

/Users/petchma/Documents/GitHub/MAT1510_Project/symbolic_deep_learning


In [5]:
import pandas as pd
default_pysr_params = dict(
    populations=30,
    procs=4, 
    model_selection="best",
)
df = pd.read_csv("data/message_"+str(0)+".csv")
df

,Unnamed: 0,e79,dx,dy,r,m1,m2
0,0,0.603133,0.543489,-0.778920,0.949788,5.785905,0.836671
1,1,0.532812,1.947129,1.091559,2.232221,5.785905,0.770745
2,2,0.595343,0.818097,-0.411026,0.915546,5.785905,0.800204
3,3,0.241558,-0.543489,0.778920,0.949788,0.836671,5.785905
4,4,0.007187,1.403639,1.870478,2.338567,0.836671,0.770745
...,...,...,...,...,...,...,...
11995,11995,-0.031735,0.491149,-0.143349,0.511641,0.163864,0.695851
11996,11996,0.023550,0.882829,-0.116672,0.890506,0.163864,1.765544
11997,11997,0.230728,0.827249,1.477600,1.693411,1.765544,0.581951
11998,11998,0.252006,-0.391680,-0.026677,0.392587,1.765544,0.695851


In [7]:
import pandas as pd
from tqdm import tqdm 
import gc 
BEST_RESULTS = []
for i in tqdm(range(0,21)):
    df = pd.read_csv("data/message_"+str(i*10)+".csv")
    y = np.array(df[df.keys()[1]].to_list())
    X = np.zeros((12000, 5))
    X[:, 0] = np.array(df[df.keys()[2]].to_list())
    X[:, 1] = np.array(df[df.keys()[3]].to_list())
    X[:, 2] = np.array(df[df.keys()[4]].to_list())
    X[:, 3] = np.array(df[df.keys()[5]].to_list())
    X[:, 4] = np.array(df[df.keys()[6]].to_list())

    model = PySRRegressor(
    model_selection="best",  # Result is mix of simplicity+accuracy
    niterations=40,
    binary_operators=["+", "*", "-", "/"],
    unary_operators=[
        "cos",
        "exp",
        "sin"
    ],
    complexity_of_operators = {"cos":2, "sin":2, "exp":2},
    # ^ Define operator for SymPy as well
    loss="loss(x, y) = (x - y)^2",
    procs= 8
    )

    model.fit(X, y)
    print("========================================================================================")
    print("========================================================================================")
    print("========================================================================================")

    print(model)

    print("========================================================================================")
    print("========================================================================================")
    print("========================================================================================")
    print(model.sympy())
    BEST_RESULTS.append([model.sympy(), model.latex()])
    model.equations_.to_csv("data/pysr_"+str(i*10)+"_1.csv")
    del model
    gc.collect()

  0%|                                          | 0/21 [00:00<?, ?it/s]/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/pysr/sr.py:1743: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(
/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/pysr/julia_helpers.py:139: UserWarning: `torch` was loaded before the Julia instance started. This may cause a segfault when runnin

Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form.
Started!

Cycles per second: 1.090e+04
Head worker occupation: 0.3%
Progress: 31 / 600 total iterations (5.167%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           6.946e-02  2.144e-07  0.23940198
3           1.762e-02  6.859e-01  (x3 * 0.112850964)
5           1.406e-02  1.127e-01  ((x3 + 0.74182695) * 0.09844473)
9           1.139e-02  5.271e-02  ((cos(0.38636923 / x3) + x3) / 10.543392)
11          1.076e-02  2.842e-02  ((cos(0.38636923 / (x3 / x1)) + x3) / 10.543392)
13          1.029e-02  2.218e-02  (((cos(0.38636923 / (x3 / x2)) - -0.074702404) + x3) / 10.543392)
15          1.007e-02  1.074e-02  (((cos((0

  5%|█▌                               | 1/21 [02:56<58:40, 176.02s/it]/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/pysr/sr.py:1743: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(
┌ Warning: You are using multithreading mode, but only one thread is available. Try starting julia with `--threads=auto`.
└ @ SymbolicRegression ~/.julia/packages/SymbolicRegression/37l4B/src

Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form.
Started!

Cycles per second: 1.370e+04
Head worker occupation: 0.3%
Progress: 39 / 600 total iterations (6.500%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           3.513e-02  -1.000e-10  -0.08721139
3           2.079e-02  2.623e-01  (x0 * -0.1266502)
5           1.319e-02  2.277e-01  ((x0 * -0.1266501) + -0.08721144)
7           1.153e-02  6.735e-02  (-0.07016762 * ((x0 + x0) + x2))
9           1.050e-02  4.645e-02  sin(((x0 + x2) * x0) * -0.07016762)
11          9.512e-03  4.957e-02  (((((x0 + x0) - x1) - 0.28280437) + x2) * -0.07016762)
13          6.158e-03  2.174e-01  ((((x2 + x0) - x1) * -0.048885576) * ((x

 10%|███▏                             | 2/21 [04:48<43:54, 138.68s/it]

...

/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/pysr/sr.py:1743: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(
┌ Warning: You are using multithreading mode, but only one thread is available. Try starting julia with `--threads=auto`.
└ @ SymbolicRegression ~/.julia/packages/SymbolicRegression/37l4B/src/SymbolicRegression.jl:414


Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form.
Started!

Cycles per second: 1.270e+04
Head worker occupation: 0.3%
Progress: 36 / 600 total iterations (6.000%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.404e-01  1.238e-07  0.01382007
3           1.028e-01  4.247e-01  (x1 * 0.31262556)
5           7.609e-02  1.506e-01  (x2 * (x1 * 0.14298177))
7           3.493e-02  3.893e-01  (((x1 * x2) + x0) / 6.8027315)
9           3.355e-02  2.008e-02  ((x0 - ((x1 * -0.8044568) * x2)) * 0.17634387)
15          2.521e-02  4.767e-02  ((((0.011640288 * x2) * (x1 - (0.58053416 * (0.076957874 - x0)))) / 0.21025275) / 0.37878492)
17          2.508e-02  2.574e-03  ((s

 95%|███████████████████████████████▍ | 20/21 [32:37<01:37, 97.90s/it]

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                        0.013827012   
	1        0.424669                                  (x1 * 0.31262556)   
	2        0.179167                            ((x0 + x1) * 0.2448429)   
	3        0.360756                     (((x1 * x2) + x0) / 6.8027315)   
	4        0.175703     (((x0 + (x1 / 0.58761996)) * 0.08321018) * x2)   
	5        0.009514  (sin((x0 + (x1 / 0.58761996)) * 0.08508039) * x2)   
	6        0.535466  (((x1 + x0) + x1) / ((3.2377455 / (x2 - 0.8402...   
	7        0.125740  (((x1 + x0) + x1) / ((3.2377455 / (x2 - 0.9977...   
	8  >>>>  0.154704  (((x1 + x0) + x1) / ((((4.3860326 - 1.3925356)...   
	
	       loss  complexity  
	0  0.240445           1  
	1  0.102838           3  
	2  0.071867           5  
	3  0.034929           7  
	4  0.024579           9  
	5  0.024116          11  
	6  0.008264          13  
	7  0.006427          1

FileNotFoundError: [Errno 2] No such file or directory: 'data/message_200.csv'

In [9]:
ep = 0
for el in BEST_RESULTS:
    print("<p> Epoch: ",str(ep*10),"\(",el[1], "\) </p>")
    ep+=1

<p> Epoch:  0 \( 0.0981 x_{3} + 0.0981 \cos{\left(\frac{0.285 x_{2}}{x_{3}} \right)} \) </p>
<p> Epoch:  10 \( \left(- 0.0551 x_{0} - 0.0678\right) \left(- 0.601 x_{1} + x_{2}\right) \) </p>
<p> Epoch:  20 \( \left(0.166 - \frac{0.167}{x_{2}}\right) \left(x_{0} + x_{1}\right) + 0.0293 \) </p>
<p> Epoch:  30 \( \sin{\left(- \frac{0.0513 x_{0}}{x_{2}} + 0.0513 x_{2} \left(x_{0} + x_{1}\right) + 0.0262 \right)} \) </p>
<p> Epoch:  40 \( \sin{\left(0.0736 x_{1} + 0.0736 \left(x_{0} + x_{1}\right) \left(x_{2} - \frac{1.14}{x_{2}}\right) \right)} + 0.0455 \) </p>
<p> Epoch:  50 \( 0.219 \left(x_{0} + 1.45 x_{1} + 0.481\right) \cos{\left(\frac{1.36}{x_{2}} \right)} \) </p>
<p> Epoch:  60 \( \frac{x_{0} + 1.54 x_{1} + 0.608}{1.53 + \frac{6.27}{x_{2} - 0.650}} \) </p>
<p> Epoch:  70 \( \left(0.517 x_{0} + x_{1} + 0.424\right) e^{- \frac{2.32}{x_{2}}} \) </p>
<p> Epoch:  80 \( 0.190 x_{0} + 0.789 x_{1} - \frac{0.789 x_{1}}{x_{2}} \) </p>
<p> Epoch:  90 \( \left(x_{0} + 2 x_{1}\right) \left(0.097

In [ ]:
df = pd.read_csv("data/pysr_190.csv")
key = "0.377398495086972*x0 + 0.65605164*x1 - 0.65605164*(0.5687067*x0 + x1)/x2"
li = df["equation"].to_list()
for i in range(len(li)):
    print(li[i])
    if li[i]== key :
        print(i)